In [4]:
from dev_spark_session import DevSparkSession 
from get_stock_data import GetStockData

stockdata = GetStockData() 
spark = DevSparkSession().spark


ERROR StatusLogger Reconfiguration failed: No configuration found for '5ffd2b27' at 'null' in 'null'
ERROR StatusLogger Reconfiguration failed: No configuration found for 'Default' at 'null' in 'null'


:: loading settings :: url = jar:file:/Users/PC/Desktop/VS%20Code%20Repositories/azure-stock-market/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/PC/.ivy2/cache
The jars for the packages stored in: /Users/PC/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-59715ef9-9eb7-4684-b711-74dd9eb2fb1f;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 75ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 54551)
Traceback (most recent call last):
  File "/Users/PC/.pyenv/versions/3.11.11/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/PC/.pyenv/versions/3.11.11/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/PC/.pyenv/versions/3.11.11/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/PC/.pyenv/versions/3.11.11/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/.venv/lib/python3.11/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/.venv/lib/python3.11

In [5]:
from pyspark.sql.functions import col,explode,sha2,concat_ws,trim, lit

### Bronze -> Silver

In [6]:
## create table 
silver_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table"  # Target location for Silver Delta table


# spark.sql( f"""
#     CREATE TABLE my_delta_table (
#         Symbol STRING,
#         ExchangeName STRING,
#         Currency STRING,
#         Type STRING,
#         ExchangeTimeZone STRING,
#         Volume STRING,
#         High STRING,
#         Low STRING,
#         Close STRING,
#         Open STRING,
#         Date STRING,
#         __CurrentFlag BOOLEAN,
#         __DeletedFlag BOOLEAN,
#         __ActivationDateTime TIMESTAMP,
#         __DeactivationDateTime TIMESTAMP,
#         __lastmodified TIMESTAMP,
#         __HashKey STRING,
#         __HashValue STRING
# )
# USING DELTA
# LOCATION  '{silver_path}';          
#         """)


## re-create table 


truncate_schema = spark.read.format("delta").load(silver_path).schema

truncate_silver = spark.createDataFrame([], schema= truncate_schema)
truncate_silver.write.format("delta").mode("overwrite").save(silver_path)



25/04/14 20:19:18 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
df_history = spark.sql(f"""
    DESCRIBE HISTORY delta.`{silver_path}` 
    LIMIT 1
""")

df_history = df_history.select("operationMetrics").head()[0]


audit =   { "outputrows" :  df_history["numOutputRows"],
            "inserted": df_history["numTargetRowsInserted"],
            "updated": df_history["numTargetRowsUpdated"],
            "deleted": df_history["numTargetRowsDeleted"] }
        

audit = { key : value for key, value  in audit.items() if value != '0' }

audit

KeyError: 'numTargetRowsInserted'